In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.llms import DeepInfra
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:
drive.mount('/content/drive/')
with open('/content/drive/My Drive/deepinfrakey.txt') as f:
    os.environ["DEEPINFRA_API_TOKEN"] =  f.read().strip()
with open('/content/drive/My Drive/opai_key.txt') as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
directory_path = '/content/drive/My Drive/HowToPlayWeiss'
chunk_size = 1000
chunk_overlap = 200
all_chunks = []

for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # Process only .txt files
        file_path = os.path.join(directory_path, filename)

        # Load the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
        start = 0
        while start < len(data):
            end = min(start + chunk_size, len(data))
            chunk = data[start:end]
            all_chunks.append(chunk)
            start += (chunk_size - chunk_overlap)

print(f"Total chunks: {len(all_chunks)}")

Total chunks: 1044


In [9]:
from langchain.schema import Document
embeddings = OpenAIEmbeddings()
all_documents = [Document(page_content=chunk) for chunk in all_chunks]
vectorstore = FAISS.from_documents(all_documents, embedding=embeddings)

In [39]:
llm = DeepInfra(model_id="meta-llama/Meta-Llama-3-70B-Instruct")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [40]:
query = """
Consider the effect: 【AUTO】 [Put 1 card from your hand into your waiting room] When this card is placed on the stage from your hand, you may pay the cost. If you do, reveal the top card of your deck, choose 1 level X or lower character in your waiting room, and return it to your hand. X is equal to the level of the revealed card. (Climax are regarded as level 0. Return the revealed card to its original place).
When might a player want to or want not to activate this effect? What zones of their board should they check before deciding whether not to activate it? """
result = conversation_chain({"question": query})
answer = result["answer"]
answer

''

In [41]:
query = """You are an expert at weiss schawrz.
You are playing the set Hololive.
It is your main phase . You have placed card \'Pavolia Reine, Time Just for the Two... \ on stage.
You are choosing whether it is in your favor to activate the effect:
【AUTO】 [Put 1 card from your hand into your waiting room] When this card is placed on the stage from your hand, you may pay the cost. If you do, reveal the top card of your deck, choose 1 level X or lower character in your waiting room, and return it to your hand. X is equal to the level of the revealed card. (Climax are regarded as level 0. Return the revealed card to its original place)

Your deck runs 10 level 3 cards, 8 cx cards, 12 level 1 cards, and 20 level 0 characters(climax cards are NOT characters).
You stage is
Center left: none
Center Middle: Pavolia Reine, Time Just for the Two...
Center right: none
Back left: none
Back right: none
You hand is : 2 x Miko Sakura, Barely Awake Eye(Level 3), 1 x Calliope Mori, A Step Towards the Future(Level 1), nothing else
Your waiting room is: 1 x Miko Sakura, Barely Awake Eye(Level 3)
Your deck has 32 cards in it
Your stock has 0 cards in it
Your memory has 0 cards in it
The first character of the answer should be a definitive 1 for Yes or 0 for No followed by explanation if neede
"""
result = conversation_chain({"question": query})
answer = result["answer"]
answer

' The effect does not allow you to retrieve a level 0 character. Climaxes are considered level 0 for the purposes of this effect.'

In [21]:
def clockPhase(series,level,clock,oppclevel,deckComp,deckColor,stage,hand,wr,mem,deck,stock):
  query = """
  You are an expert at weiss schawrz.
You are playing the set {series}. It is your clock phase, you are deciding whether to clock and if so, which card should be clocked. When deciding whether to clock,
make sure to consider which color you need in your clock or level in order to play the characters, events, and climaxes in your hand. Remember that in order to play a climax card or a character/event
level 1 or above you will need the color of the card in your clock or level. Consider clocking cards that can't be played the turn of that can allow you to play other cards,
if a card has a cxcombo effect that can be used that turn then try to clock so that the card is playable. consider the effects of all cards.
Your level is {numLevel}. the cards inside your level zone are: {level}
There are {numClock} cards in your clock zone. The card(s) are: {clock}
Your opponent is level {oppLevel} with {oppClock} cards in clock
Your deck runs {l3} level 3 cards, {cx} cx cards, {l1} level 1 cards, and {l0} level 0 characters(climax cards are NOT characters).
The color of your deck is : 10 red cards, 25 yellow cards, 10 blue cards, 5 green cards
Center left: none
Center Middle: none
Center right: none
Back left: none
Back right: none
Your hand is: {hand}
Your waiting room is: {wr}
Your deck has {deck} cards in it
Your stock has {stock} cards in it
Your memory is: {mem}
The response should be a definitive 1 or 0 for whether to clock and if yes, a number of the position in hand for the card to be clocked(starting at 0 left to right). Followed by an explanation if nessecary.
  """
  query = query.format(series = series, numLevel = len(level), level = level, numClock = len(clock), clock = clock, oppLevel = oppclevel/10,oppClock = oppclevel%10, l3 = deckLevels[3],l2 = deckLevels[2],l1 = deckLevels[1],l0 = deckLevels[0],cx = deckLevels[4],hand = hand, wr = wr, mem = mem, deck = deck, stock = stock)
  result = conversation_chain({"question": query})
  answer = result["answer"]
  answer
  return answer

In [42]:
series = "hololive" #set/series the bot is playing
level = ["Miko Sakura, Barely Awake Eye(Level 3, Red)"] #list of cards in level, use len() to get level
clock = [] # list of cards in clock
oppcl = 10 # int representation of opponent's level/clock. first character is level, second is clock. get each by using /10 or %10
deckLevels = [20,12,2,8,8] # list representation of deck level distribution. Left to right is #level 0s(not including climaxes), #level 1s, #level 2s, #level 3s,  #climaxes
deckColors = [10,25,10,5] # color distribution. left to right is Red, Yellow, Blue, Green
stage = ['none','none','none','none','none'] # stage state. LeftCenter, MiddleCenter, RightCenter, LeftBack, RightBack respectively
hand =  ["A Step Towards the Future, Mori Calliope(Level 1, Yellow)","A Step Towards the Future, Mori Calliope(Level 1, Yellow)", "#FestivalDrawings, Natsuiro Matsuri (Level 3, Yellow)","Yamada Hermione, Minato Aqua(Level 0, Blue)","Yamada Hermione, Minato Aqua(Level 0, Blue)"] #List of cards in hand. Don't remove duplicates
wr = ["Pavolia Reine, Time Just for the Two..."] # list of cards in waiting room
deck = 40 # int of number of cards in deck
stock = 2 # int of number of cards in stock
mem = [] #list of cards in memory



print(clockPhase(series,level,clock,oppcl,deckLevels,deckColors,stage,hand,wr,deck,stock,mem))

  I don't know. The context provided does not give enough information about the current state of your board and hand to make a recommendation.
